# Data Warehouse Medicare Texas QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

In [3]:
connection.close()

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [3]:
query = '''drop table if exists qa_reporting.dw_mcrt_mbr_enrl_yearly;
create table qa_reporting.dw_mcrt_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_mcrt_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.mcrt_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [5]:
with connection.cursor() as cursor:
    
      query = '''
update qa_reporting.dw_mcrt_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.medicare_texas_counts b
where data_source = 'mcrt'
and a.calendar_year = b.year
and a.table_src = 'medicare_texas.' || b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as(
select year::int, bene_id, t.month_year_id as enrolled_month
            from medicare_texas.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
),
enrl_my as (
    select 'medicare_texas.mbsf_abcd_summary' table_name, year, count(enrolled_month) as mm
    from enrl_months
    group by 2
)
update qa_reporting.dw_mcrt_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'mcrt'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

In [6]:
df = pd.read_sql('''select * 
from qa_reporting.dw_mcrt_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,mcrt,2018,medicare_texas.mbsf_abcd_summary,4262436,4262436,4262436,4284529,-22093,0.0,48608033,48871117,-263084,0.0,2023-05-30
1,mcrt,2019,medicare_texas.mbsf_abcd_summary,4391499,4391499,4391499,4411405,-19906,0.0,50136099,50396299,-260200,0.0,2023-05-30
2,mcrt,2017,medicare_texas.mbsf_abcd_summary,4173983,4173983,4173983,4194289,-20306,0.0,47618583,47866081,-247498,0.0,2023-05-30
3,mcrt,2015,medicare_texas.mbsf_abcd_summary,3928931,3928931,3928931,3949215,-20284,0.0,44757434,45003185,-245751,0.0,2023-05-30
4,mcrt,2016,medicare_texas.mbsf_abcd_summary,4049784,4049784,4049784,4069556,-19772,0.0,46210126,46448004,-237878,0.0,2023-05-30
5,mcrt,2020,medicare_texas.mbsf_abcd_summary,4523458,4523458,4523458,4538440,-14982,0.0,51632803,51868937,-236134,0.0,2023-05-30
6,mcrt,2014,medicare_texas.mbsf_abcd_summary,3803218,3803218,3803218,3822796,-19578,0.0,43306304,43529167,-222863,0.0,2023-05-30


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [7]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2014,-222863,0.0,-19578,0.0
2015,-245751,0.0,-20284,0.0
2016,-237878,0.0,-19772,0.0
2017,-247498,0.0,-20306,0.0
2018,-263084,0.0,-22093,0.0
2019,-260200,0.0,-19906,0.0
2020,-236134,0.0,-14982,0.0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column

In [8]:
query = '''with mcrn_gen_cd as (
    select distinct year::int, bene_id, sex_ident_cd
    from medicare_texas.mbsf_abcd_summary a
),
mcrn_gen as (
    select year, c.gender_cd, count(*) gender_count
    from mcrn_gen_cd m
    left join reference_tables.ref_gender c
    on c.data_source = 'mcr'
   and c.gender_cd_src = m.sex_ident_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from mcrn_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2014,F,2059571,2068579,-9008,0.435468
1,2014,M,1743646,1754216,-10570,0.602548
2,2014,U,1,1,0,0.000000
3,2015,F,2125231,2134520,-9289,0.435180
4,2015,M,1803699,1814694,-10995,0.605887
5,2015,U,1,1,0,0.000000
6,2016,F,2188211,2197429,-9218,0.419490
7,2016,M,1861571,1872125,-10554,0.563744
8,2016,U,2,2,0,0.000000
9,2017,F,2253351,2262906,-9555,0.422245


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference percent should be <= 1%.

In [9]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, ent.plan_type, t.month_year_id
    from medicare_texas.mbsf_abcd_summary a
    cross join lateral (values (year || '01', a.mdcr_entlmt_buyin_ind_01, a.mdcr_status_code_01), (year || '02', a.mdcr_entlmt_buyin_ind_02, a.mdcr_status_code_02),
                        (year || '03', a.mdcr_entlmt_buyin_ind_03, a.mdcr_status_code_03), (year || '04', a.mdcr_entlmt_buyin_ind_04, a.mdcr_status_code_04), (year || '05', a.mdcr_entlmt_buyin_ind_05, a.mdcr_status_code_05),
                        (year || '06', a.mdcr_entlmt_buyin_ind_06, a.mdcr_status_code_06), (year || '07', a.mdcr_entlmt_buyin_ind_07, a.mdcr_status_code_07), (year || '08', a.mdcr_entlmt_buyin_ind_08, a.mdcr_status_code_08),
                        (year || '09', a.mdcr_entlmt_buyin_ind_09, a.mdcr_status_code_09), (year || '10', a.mdcr_entlmt_buyin_ind_10, a.mdcr_status_code_10), (year || '11', a.mdcr_entlmt_buyin_ind_11, a.mdcr_status_code_11),
                        (year || '12', a.mdcr_entlmt_buyin_ind_12, a.mdcr_status_code_12))
    t(month_year_id, mcdcr_enrlmt, enrollment_status)
    join reference_tables.ref_medicare_entlmt_buyin ent 
    on ent.buyin_cd = t.mcdcr_enrlmt
    where t.enrollment_status in ('10','11','20','21','31')
),
mcrn_agg_enroll as (
    select year, bene_id, plan_type, max(month_year_id) my, count(*)
    from mcrn_enroll
    group by 1,2,3
),
mcrn_grp_enroll as (
    select *, row_number() over(partition by bene_id, year order by count desc, my desc) as my_grp
    from mcrn_agg_enroll
),
mcrn_plans as (
    select year, plan_type, count(*) plan_count
    from mcrn_grp_enroll 
    where my_grp = 1
    group by 1,2
),
dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from mcrn_plans b
join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2014,A,302284,321792,-19508,6.062301
1,2014,B,11239,11891,-652,5.483139
2,2014,AB,3489695,3488812,883,0.025309
3,2015,AB,3601566,3600227,1339,0.037192
4,2015,B,12869,13532,-663,4.899497
5,2015,A,314496,335208,-20712,6.178850
6,2016,A,328831,348431,-19600,5.625217
7,2016,AB,3707502,3706284,1218,0.032863
8,2016,B,13451,14188,-737,5.194531
9,2017,A,344969,365301,-20332,5.565821


In [10]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2014    0.025309
 2015    0.037192
 2016    0.032863
 2017    0.025927
 2018    0.034582
 2019    0.050048
 2020    0.079675
 Name: plan_diff_percent, dtype: float64,
 year
 2014    6.062301
 2015    6.178850
 2016    5.625217
 2017    5.565821
 2018    6.556353
 2019    6.513286
 2020    4.434001
 Name: plan_diff_percent, dtype: float64)

In [11]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
2,2014,AB,3489695,3488812,883,0.025309
10,2017,AB,3815541,3814552,989,0.025927
7,2016,AB,3707502,3706284,1218,0.032863
12,2018,AB,3890627,3889282,1345,0.034582
3,2015,AB,3601566,3600227,1339,0.037192
15,2019,AB,4000134,3998133,2001,0.050048
20,2020,AB,4131315,4128026,3289,0.079675
19,2020,B,11280,11734,-454,3.869098
18,2020,A,380863,398534,-17671,4.434001
4,2015,B,12869,13532,-663,4.899497


In [13]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df['percent_diff'] = 100. * plan_types_agg_df['plan_diff'] / plan_types_agg_df['src_plan_count'] 
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\3\ipykernel_8068\1871058508.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2014,3803218,3822495,-19277,-0.504304
2015,3928931,3948967,-20036,-0.507373
2016,4049784,4068903,-19119,-0.469881
2017,4173983,4194036,-20053,-0.478131
2018,4262436,4284273,-21837,-0.509701
2019,4391499,4411218,-19719,-0.447019
2020,4523458,4538294,-14836,-0.326907


## State

In [14]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, case when state_cd is null then 'XX' else state_cd end as state
    from medicare_texas.mbsf_abcd_summary m
    left outer join reference_tables.ref_medicare_state_codes e 
     on e.medicare_state_cd = m.state_code   
),
mcrn_states as (
    select year, state, count(*) state_count
    from mcrn_enroll m
    group by 1,2
), 
dw_plans as (
    select year, case when state is null then 'XX' else state end as state, count(*) state_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from mcrn_states b
full join dw_plans a
on a.year = b.year
and a.state = b.state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2014,MS,1949,1956,-7,0.357873
1,2014,PA,2264,2279,-15,0.658183
2,2014,FL,9267,9322,-55,0.590002
3,2014,NV,2108,2124,-16,0.753296
4,2014,IL,4792,4824,-32,0.663350
...,...,...,...,...,...,...
373,2020,TX,4484967,4499945,-14978,0.332849
374,2020,PA,608,608,0,0.000000
375,2020,AL,813,813,0,0.000000
376,2020,AK,101,101,0,0.000000


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of 'XX'. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [15]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
28,2014,XX,1551,2505,-954,38.083832
135,2016,XX,2079,2420,-341,14.090909
102,2015,XX,1876,2134,-258,12.089972
20,2014,DC,128,131,-3,2.290076
31,2014,CT,670,682,-12,1.759531
...,...,...,...,...,...,...
211,2017,DC,92,92,0,0.000000
213,2017,SD,389,389,0,0.000000
214,2017,OK,6786,6786,0,0.000000
216,2018,DC,52,52,0,0.000000


Should get up to 50 states plus Puerto Rico (PR), Washington D.C. (DC), Virgin Islands (VI), and unknown (XX)

In [16]:
df['state'].unique()

array(['MS', 'PA', 'FL', 'NV', 'IL', 'MT', 'MD', 'ND', 'VA', 'HI', 'NE',
       'DE', 'NC', 'GA', 'CO', 'MN', 'ME', 'WA', 'LA', 'NH', 'DC', 'PR',
       'TX', 'NY', 'SD', 'SC', 'TN', 'AZ', 'XX', 'MI', 'NM', 'CT', 'MO',
       'OR', 'AL', 'OH', 'KY', 'VT', 'RI', 'WY', 'MA', 'IA', 'WV', 'CA',
       'UT', 'AK', 'KS', 'IN', 'ID', 'OK', 'WI', 'VI', 'NJ', 'AR'],
      dtype=object)

In [17]:
df['state'].unique().shape

(54,)

In [18]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [19]:
df[df['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
